## Install and Load libraries

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 12.6 MB/s 
     |████████████████████████████████| 4.4 MB 54.8 MB/s 
     |████████████████████████████████| 101 kB 7.7 MB/s 
     |████████████████████████████████| 212 kB 87.7 MB/s 
     |████████████████████████████████| 140 kB 58.7 MB/s 
     |████████████████████████████████| 1.1 MB 54.5 MB/s 
     |████████████████████████████████| 596 kB 78.3 MB/s 
     |████████████████████████████████| 127 kB 93.9 MB/s 
     |████████████████████████████████| 6.6 MB 76.8 MB/s 
     |████████████████████████████████| 271 kB 76.0 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 76.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing

In [2]:
from datasets import load_dataset, load_metric

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import transformers

print(transformers.__version__)

4.20.1


In [22]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = True
model_checkpoint = 'drive/MyDrive/w266/model_checkpoints_2/DistilBert/Distilbert-NQ-Trivia-cased-trained_compact_final'
batch_size = 16

In [7]:
from datasets import load_dataset
dataset_train_val = load_dataset('squad_v2')
dataset_train_test = load_dataset('squad_v2')
dataset_train_test = dataset_train_test['train']
dataset_train_test = dataset_train_test.train_test_split(test_size=0.1)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
dataset_train_val

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [9]:
dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 117287
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 13032
    })
})

In [19]:
train = dataset_train_val['train']
validation = dataset_train_val['validation']
test = dataset_train_test['test']

In [20]:
train.save_to_disk('drive/MyDrive/w266/data/SQuAD/train_master.json')
validation.save_to_disk('drive/MyDrive/w266/data/SQuAD/validation_master.json')
test.save_to_disk('drive/MyDrive/w266/data/SQuAD/test_master.json')

## Preprocessing the training data

We will use TokenizerFast and leverage a few properties for QA task, such as return offset mapping etc. 

In [23]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [24]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [15]:
# tokenizer("What is your name?", "My name is Sylvain.")

In [25]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [26]:
pad_on_right = tokenizer.padding_side == "right"

In [28]:
def prepare_train_features(examples):
    # remove white space in question section 
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # maps feature back to sample
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # map from token to character position in the original context. 
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [29]:
tokenized_train = train.map(prepare_train_features, batched=True, remove_columns=train.column_names)
tokenized_test = test.map(prepare_train_features, batched=True, remove_columns=test.column_names)

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

## Fine-tuning the model

In [30]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [31]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-SQuAD",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01
)

    # push_to_hub=True,

In [32]:
from transformers import default_data_collator

data_collator = default_data_collator

In [33]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


    # eval_dataset=tokenized_datasets["validation"],

In [34]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 132079
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8255


Epoch,Training Loss,Validation Loss
1,1.352600,1.150043


Saving model checkpoint to Distilbert-NQ-Trivia-cased-trained_compact_final-finetuned-SQuAD/checkpoint-500
Configuration saved in Distilbert-NQ-Trivia-cased-trained_compact_final-finetuned-SQuAD/checkpoint-500/config.json
Model weights saved in Distilbert-NQ-Trivia-cased-trained_compact_final-finetuned-SQuAD/checkpoint-500/pytorch_model.bin
tokenizer config file saved in Distilbert-NQ-Trivia-cased-trained_compact_final-finetuned-SQuAD/checkpoint-500/tokenizer_config.json
Special tokens file saved in Distilbert-NQ-Trivia-cased-trained_compact_final-finetuned-SQuAD/checkpoint-500/special_tokens_map.json
Saving model checkpoint to Distilbert-NQ-Trivia-cased-trained_compact_final-finetuned-SQuAD/checkpoint-1000
Configuration saved in Distilbert-NQ-Trivia-cased-trained_compact_final-finetuned-SQuAD/checkpoint-1000/config.json
Model weights saved in Distilbert-NQ-Trivia-cased-trained_compact_final-finetuned-SQuAD/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in Distilbert-NQ-

TrainOutput(global_step=8255, training_loss=1.5730405866413821, metrics={'train_runtime': 4949.9018, 'train_samples_per_second': 26.683, 'train_steps_per_second': 1.668, 'total_flos': 1.2942398099344896e+16, 'train_loss': 1.5730405866413821, 'epoch': 1.0})

In [ ]:
trainer.save_model("drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-NQ-cased-trained")

Saving model checkpoint to drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-NQ-cased-trained
Configuration saved in drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-NQ-cased-trained/config.json
Model weights saved in drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-NQ-cased-trained/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-NQ-cased-trained/tokenizer_config.json
Special tokens file saved in drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-NQ-cased-trained/special_tokens_map.json


In [ ]:
!cp -R distilbert-base-cased-finetuned-NQ drive/MyDrive/w266/model_checkpoints/DistilBert/

In [ ]:
# !cp -R test-NQ-trained drive/MyDrive/w266/model_checkpoints/DistilBert/

In [ ]:
# trainer.save_model("drive/MyDrive/w266/model_checkpoints/DistilBert/test-NQ-trained/")

In [ ]:
# !ls drive/MyDrive/w266/model_checkpoints/DistilBert/test-NQ-trained

In [ ]:
!ls drive/MyDrive/w266/model_checkpoints/DistilBert/

distilbert-base-cased-finetuned-NQ    distilBert-NQ-cased-trained
distilbert-base-uncased-finetuned-NQ


## Evaluation

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 65,  43, 126,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0], device='cuda:0'),
 tensor([ 29,  51, 126,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0], device='cuda:0'))

Consider the following sceanarios: 
- the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

By:

- score: adding the start and end logits
- hyper-parameter we call n_best_size

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

### process validation set:
- will need example id and offset mapping needed

In [ ]:
def prepare_validation_features(examples):

    # similar process as train
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # need example_id for matching
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

Map the validation set

In [ ]:
validation_features = datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["test"].column_names
)

  0%|          | 0/14 [00:00<?, ?ba/s]

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [ ]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `DistilBertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 53884
  Batch size = 16


The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing).

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

Upper limit to avoid very long answers from our considerations.

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["test"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 6.136654, 'text': 'Constantinople'},
 {'score': 5.3226137, 'text': 'The Byzantine Empire'},
 {'score': 5.2358737,
  'text': 'Constantinople ( modern - day Istanbul , which had been founded as Byzantium )'},
 {'score': 4.9988112, 'text': 'Byzantine Empire'},
 {'score': 4.571806, 'text': 'Constantinople ( modern - day Istanbul'},
 {'score': 3.7912328,
  'text': 'The Byzantine Empire , also referred to as the Eastern Roman Empire'},
 {'score': 3.7627873,
  'text': 'Eastern Roman Empire , was the continuation of the Roman Empire in the East during Late Antiquity and the Middle Ages , when its capital city was Constantinople'},
 {'score': 3.4674306,
  'text': 'Byzantine Empire , also referred to as the Eastern Roman Empire'},
 {'score': 3.289897,
  'text': 'the continuation of the Roman Empire in the East during Late Antiquity and the Middle Ages , when its capital city was Constantinople'},
 {'score': 3.283916,
  'text': 'Constantinople ( modern - day Istanbul , which had been f

Compare to ground-truth answer:

In [ ]:
datasets["test"][0]["answers"]

{'answer_start': [51], 'text': ['Eastern Roman Empire']}

Our model picked the right as the most likely answer!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [ ]:
import collections

examples = datasets["test"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

## Post-processing function
- predict the impossible answer when that score is greater than the score of the best non-impossible answer

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["test"], validation_features, raw_predictions.predictions)

Post-processing 13694 example predictions split into 53884 features.


  0%|          | 0/13694 [00:00<?, ?it/s]

Use the squad v2 metric for evluation

In [ ]:
metric = load_metric("squad_v2")

## Final Evaluation

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["test"]]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 46.53382275825905,
 'HasAns_f1': 54.12521405061787,
 'HasAns_total': 9535,
 'NoAns_exact': 78.04760759798029,
 'NoAns_f1': 78.04760759798029,
 'NoAns_total': 4159,
 'best_exact': 56.11946838031255,
 'best_exact_thresh': 0.0,
 'best_f1': 61.40528085093065,
 'best_f1_thresh': 0.0,
 'exact': 56.10486344384402,
 'f1': 61.39067591446194,
 'total': 13694}